<h1>Peer Graded Assignment - Segmentation and Clustering in the neighbourhoods in Toronto

<h2> A] Scraping and cleaning the data from Wikipedia

<h3>1) Reading Toronto data which was extracted into a csv file from Wikipedia

In [1]:
import numpy as np # library to handle data in a vectorized manner
import config

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

ModuleNotFoundError: No module named 'geopy'

In [ ]:
df = pd.read_csv('/Users/Rohit/Documents/Foursquare_PG/Toronto_data.csv')

In [ ]:
df.head()
df.shape

<h3> 2) Dropping cells which have "Not assigned" values

In [ ]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True) 
df.shape

In [ ]:
pd.set_option('display.max_rows', None)
df.head()
print(df['Postal Code'].value_counts().head())

In [ ]:
df.dtypes

In [ ]:
df.head()

<h3>3) Combining rows with same postal codes by using comma seperators for Neighborhoods

In [ ]:
df = df.groupby('Postal Code').agg({'Borough':'first', 
                             'Neighborhood': ', '.join, 
                              }).reset_index()

In [ ]:
#pd.set_option('display.max_rows', None)
df.head()

In [ ]:
#df2 = pd.DataFrame({ "Postal Code":['A12'], 
#                    "Borough":['Scarborough'],
#                    "Neighborhood":'Not assigned'})

In [ ]:
#df3 = df.append(df2, ignore_index=True)

In [ ]:
#df3.shape

<h3> 4) Replacing Neighborhoods with borough names in case it is "Not assigned"

In [ ]:
df['Neighborhood'].mask(df['Neighborhood'] == 'Not assigned', df['Borough'], inplace=True)
df.head()

<h3> 5) Shape of the dataframe after cleaning

In [ ]:
df.shape

In [ ]:
'''
Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. 
Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/
'''

<h2> B] Dataframe after adding the latitudes and longitudes

<h3>1) Reading the coordinates file

In [ ]:
cord_path = '/Users/Rohit/Desktop/Capstone_projs/Geospatial_Coordinates.csv'
df_cord = pd.read_csv(cord_path)
df_cord.head()

<h3>2) Merging coordinates file with the dataframe

In [ ]:
Toronto_merged = df_cord

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(df.set_index('Postal Code'), on='Postal Code')
Toronto_merged = Toronto_merged[['Postal Code','Borough','Neighborhood','Latitude', 'Longitude']]
Toronto_merged.head()

<h2> C] Exploring and Clustering Datasets

<H3> 1) Filtering data so that the neighborhood has the word "Toronto"

In [ ]:
Toronto_filtered = Toronto_merged[Toronto_merged['Borough'].str.contains("Toronto")]

In [ ]:
Toronto_filtered.head()

<b>Defining credentials for Foursquare

In [ ]:
#Rohit
CLIENT_ID = config.CLIENT_ID # your Foursquare ID
CLIENT_SECRET = config.CLIENT_SECRET # your Foursquare Secret
VERSION = config.VERSION # Foursquare API version
LIMIT = config.LIMIT

<h3> 2) Define a function for getting json file and converting it to a dataframe with venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        results = results["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3> 3) Feed in Toronto filtered dataset in the function

In [ ]:
Toronto_venues = getNearbyVenues(names=Toronto_filtered['Neighborhood'],
                                   latitudes=Toronto_filtered['Latitude'],
                                   longitudes=Toronto_filtered['Longitude']
                                  )

In [ ]:
Toronto_venues.head()

<h3> 4) Checking size of the resulting dataframe

In [ ]:
print(Toronto_venues.shape)
Toronto_venues.head()

<h3> 5) Let's check how many venues were returned for each neighborhood

In [ ]:
Toronto_venues.groupby('Neighborhood').count().head()

<h3> 6) Finding out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

<h3>7) Using one hot encoding for Venues

In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

In [ ]:
Toronto_onehot.shape

<h3>8) Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

In [ ]:
Toronto_grouped.shape

<h3>9) Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    
    #print(temp)
    temp.columns = ['venue','freq']
    #print(temp)
    temp = temp.iloc[1:]
    
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

<h3>10) Let's put that into a *pandas* dataframe

In [ ]:
#Function sorted values
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<h3>11) Clustering Neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

<h3> 12) Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_filtered

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

<h3>13) Finally, let's visualize the resulting clusters

In [ ]:
# create map
latitude = 43.651070
longitude = -79.347015
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>14) Examining Clusters

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]